In [122]:
import numpy as np
import matplotlib.pyplot as plt
import binpacking
from ortools.linear_solver import pywraplp
import sys
import time
#np.set_printoptions(threshold = sys.maxsize)
np.set_printoptions(threshold=np.inf)

np.random.seed(0)

In [89]:
def init(B,I,N_design):
    
    #Data initialization

    Budget = np.loadtxt('Budget.txt', usecols=range(N_design))
    Data = np.loadtxt('Data.txt', usecols=range(I))
    ExpectedGain = np.loadtxt('Expected.txt',usecols=range(N_design))
    varNPV = np.zeros((7,6))
    Pos = np.zeros((I,N_design))
    one_beta = Data[18:24,:]
    P_i_a = Data[1,:]
    alpha_i = Data[3,:]
    n__trials = Data[16,:]
    r = np.zeros((I,N_design))

    #calculating PoSij
    for i in range(I):
        for j in range(N_design):
            Pos[i][j] = (one_beta[j][i]**n__trials[i])*P_i_a[i] + (alpha_i[i]**n__trials[i])*(1-P_i_a[i])
            if Pos[i][j] != 0: r[i][j] = (ExpectedGain[i][j]+Budget[i][j]*(1-Pos[i][j]))/Pos[i][j]
            else:r[i][j] = 0
    #calculating varNPV        
    for i in range(I):
        for j in range(N_design):
            varNPV[i][j] = np.sum((ExpectedGain-r[i][j])**2)/(I*N_design-1)

    #lambda = max(e_ij)/max(varNPV)
    lmbda = np.max(ExpectedGain)/np.max(varNPV)
    #lmbda = 0
    return Budget, varNPV, ExpectedGain, lmbda, P_i_a

I = 7
B_Total = 150
N_design = 6
Budget,varNPV,ExpectedGain,lmbda,p_i_a = init(B_Total,I,N_design)

class drug:
      def __init__(self,index,NumberOfDesigns,cost,ExpectedGain,p_i_a):
            self.N_design = NumberOfDesigns
            self.i = index
            self.eGain = ExpectedGain[index][:]
            self.cost = cost[index][:]
            self.p_i_a = p_i_a


In [124]:
drug_list = []
for i in range(I):
    drug_list.append(drug(i,N_design,Budget,ExpectedGain,p_i_a[i]))
    
def master_func(J,opt_dec_list,I):
    f = open('result.txt', 'w')
    for i in range(I,-1,-1):
        opt_dec_list = Update_Opt_Decs(i,opt_dec_list)
        print(*opt_dec_list)
        f.write(str(i))
        f.write("\n \n")
    return opt_dec_list

def Update_Opt_Decs(i,opt_dec_list):
    eG_vec = find_opt_decs(i,opt_dec_list)
    opt_dec_list[i][:,1] = eG_vec
    #print(opt_dec_list)
    return opt_dec_list

def find_opt_decs(i,opt_dec_list):
   
    B_vec = opt_dec_list[i][:,0]
    eGain_vec = np.zeros(len(B_vec))
    for r in range(len(B_vec)):
        eGain_vec[r] = bellman(i, B_vec[r], drug_list)
         
    return eGain_vec  

def Jcal(inDrug,B):
    j = []
    for i in range(inDrug.N_design):
        if inDrug.cost[i]<=B:
            j.append(i)
    return j

In [108]:
def bellman(i,B,drug_list):
    drug = drug_list[i]
    Jj = Jcal(drug,B)
    if i == (len(drug_list)-1):
        temp = []
        for k in Jj:
            temp.append(drug.eGain[k])
        return drug.p_i_a * max(temp) 
    else:
        temp = []
        for k in Jj:
            temp.append(drug.eGain[k]+bellman(i+1,B-drug.cost[k],drug_list))

        epsilon_i = drug.p_i_a * max(temp) + (1-drug.p_i_a) * bellman(i+1,B,drug_list)
    return epsilon_i

In [125]:
opt_dec_list = []
data_frame = np.zeros((10,2))

temp = np.linspace(0,150,num=10)
data_frame[:,0] = temp

for i in range(I):
    opt_dec_list.append(data_frame)

opt_dec_list =  master_func(0,opt_dec_list,I-1)

with open('result.txt', 'w') as f:
    for i in opt_dec_list:
        #print(i,"\n\n")
        f.write(str(i))
        f.write("\n \n")

[[  0.           0.        ]
 [ 16.66666667 647.7       ]
 [ 33.33333333 680.8       ]
 [ 50.         680.8       ]
 [ 66.66666667 680.8       ]
 [ 83.33333333 680.8       ]
 [100.         680.8       ]
 [116.66666667 680.8       ]
 [133.33333333 680.8       ]
 [150.         680.8       ]] [[  0.           0.        ]
 [ 16.66666667 647.7       ]
 [ 33.33333333 680.8       ]
 [ 50.         680.8       ]
 [ 66.66666667 680.8       ]
 [ 83.33333333 680.8       ]
 [100.         680.8       ]
 [116.66666667 680.8       ]
 [133.33333333 680.8       ]
 [150.         680.8       ]] [[  0.           0.        ]
 [ 16.66666667 647.7       ]
 [ 33.33333333 680.8       ]
 [ 50.         680.8       ]
 [ 66.66666667 680.8       ]
 [ 83.33333333 680.8       ]
 [100.         680.8       ]
 [116.66666667 680.8       ]
 [133.33333333 680.8       ]
 [150.         680.8       ]] [[  0.           0.        ]
 [ 16.66666667 647.7       ]
 [ 33.33333333 680.8       ]
 [ 50.         680.8       ]
 [ 66.66666